# Web Scraping Facebook with Selenium

Download your <b>entire</b> image gallery with a <b>100% automation</b>
<br>
Targeting:
<ul>
    <li>images uploaded by yourself</li>
    <li>images you were tagged in, uploaded by others</li>
</ul>

### Download ChromeDriver
You can either download latest stable release of ChromeDriver from:
https://chromedriver.chromium.org/
<br>
### OR!! Use webdriver_manager Istead
Or, you can install a library named webdriver_manager that will install it for you.
<br>
https://pypi.org/project/webdriver-manager/
<br>
<br>
Thank you so much to <b>Avishek2020</b> for pointing this INREDIBLE alternative on Github:
<br>
https://github.com/MariyaSha/TwitterBot/issues/1

## Install Requirements

In [1]:
!pip install webdriver_manager

In [2]:
!pip install wget
!pip install selenium

### Import Selenium & time

In [3]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

### Step 1: Disable Alerts/Notifications

!! Using code by <b>pythonjar</b> of <b>Stackoverflow</b> !!
<br>
Please visit this thread for more details:
<br>
https://stackoverflow.com/questions/41400934/handle-notifications-in-python-selenium-chrome-webdriver

In [4]:
#code by pythonjar, not me
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

### Step 2: Log into your personal Facebook account
MAKE SURE to replace <b><i>my_username</i></b> and <b><i>my_password</i></b> with your own unique values


In [5]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

#open the webpage
driver.get("http://www.facebook.com")

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("my_username")
password.clear()
password.send_keys("my_password")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#We are logged in!

### Step 3: Extract All the Photos

!! Update <b><i>goldie.may.750</i></b> to your own unique profile address !!
<ul>
    <li>Create an empty list <b><i>(images)</i></b></li>
    <li>Loop over all the photos you've ever uploaded (<b><i>photos_all</i></b>)</li>
    <li>Loop over all the photos you were ever tagged in by other users(<b><i>photos</i></b>)</li>
    <li>Scroll to the very bottom of the page (or to a certain range)</li>
    <li>Extract links to photos</li>
    <li>Follow the link and extract store the image inside <b><i>(images)</i></b></li>
</ul>

In [6]:
#wait 5 seconds to allow your new page to load
time.sleep(5)
images = [] 

#itterate over both uploaded and tagged images respectively
for i in ["photos_by", "photos_of"]:
    # ************************************************
    # !! change goldie.may.750 to your own address !!
    # ************************************************
    driver.get("https://www.facebook.com/goldie.may.750/" + i + "/")
    time.sleep(5)
    
    #scroll down
    #increase the range to sroll more
    #example: range(0,10) scrolls down 650+ images
    for j in range(0,1):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)

    #target all the link elements on the page
    anchors = driver.find_elements(By.TAG_NAME, 'a')
    anchors = [a.get_attribute('href') for a in anchors]
    #narrow down all links to image links only
    anchors = [a for a in anchors if str(a).startswith("https://www.facebook.com/photo")]
    
    print('Found ' + str(len(anchors)) + ' links to images')
    
    #extract the [-1]st image element in each link
    for a in anchors:
        driver.get(a) #navigate to link
        time.sleep(5) #wait a bit
        img = driver.find_elements(By.TAG_NAME, "img")
        images.append(img[-1].get_attribute("src")) #may change in future to img[?]

print('I scraped '+ str(len(images)) + ' images!')

Found 3 links to images
Found 0 links to images
I scraped 3 images!


### Make sure you got the right images!

Before we move on, please check if the first URL leads to the first image you wanted to scrape. If not, please addust the index of your <b>img</b> variable from 1 to what's currently relevant (in the previous cell)

In [7]:
 images

['https://scontent.fcxh2-1.fna.fbcdn.net/v/t1.6435-9/140352504_100665925362090_559225914483352421_n.jpg?_nc_cat=105&cb=99be929b-3346023f&ccb=1-7&_nc_sid=0debeb&_nc_ohc=u1mSkW-EenIAX_xVC59&_nc_ht=scontent.fcxh2-1.fna&oh=00_AfCY4IEcAr48anB2ivlPmzVj8BnMxy-UtupBR8kvcCU80g&oe=64B44B63',
 'https://scontent.fcxh2-1.fna.fbcdn.net/v/t1.6435-9/140134143_100665918695424_4499568289102186586_n.jpg?_nc_cat=106&cb=99be929b-3346023f&ccb=1-7&_nc_sid=0debeb&_nc_ohc=VTv_6lWdLLgAX9eHj3F&_nc_ht=scontent.fcxh2-1.fna&oh=00_AfDQ0CtCQJr4iboENyEHA7_6KjcVDZ6IUrrfvklBCWuz_w&oe=64B459BA',
 'https://static.xx.fbcdn.net/rsrc.php/v3/yA/r/AMeFSQ6eX1J.png']

### Step 4: Create a new directory in the current folder


using OS path

In [8]:

import os

path = os.getcwd()
path = os.path.join(path, "FB_SCRAPED")

#create the directory
os.mkdir(path)

### Step 5: Save all the photos in the new directory

Using wget

In [9]:
import wget

#download images
counter = 0
for image in images:
    save_as = os.path.join(path, str(counter) + '.jpg')
    wget.download(image, save_as)
    counter += 1

100% [..................................................................................] 330 / 330